In [1]:
import gspread, gspread_dataframe
from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd
import re

def num_to_col_letters(num) :
    letters = ''
    while num:
        mod = (num - 1) % 26
        letters += chr(mod + 65)
        num = (num - 1) // 26
    return ''.join(reversed(letters))

In [2]:
def connect_to_sheets(spreadsheet='GemeinsameBilanzierung_16_17', sheetname='') :
    
    if type(spreadsheet) is str : 
        scope = ['https://spreadsheets.google.com/feeds']
        creds = ServiceAccountCredentials.from_json_keyfile_name('client-secret.json', scope)
        client = gspread.authorize(creds)
        spreadsheet = client.open(spreadsheet)

    # Find a workbook by name and open the first sheet
    # Make sure you use the right name here.
    if sheetname == '' :
        return spreadsheet
    else :
        worksheet = spreadsheet.worksheet(sheetname)
        return worksheet

In [4]:
res = { # re for at all valid
               'valid_range' : r'^[A-Z]+\d*(:[A-Z]+\d*)?$',
               # re for complete range
               'normal_range' : r'^[A-Z]+\d+:[A-Z]+\d+$',
               # re for single cell
               'single_cell' : r'^[A-Z]+\d+$',
               # re for only letters
               'only_cols' : r'^[A-Z]+:[A-Z]+$',
               # re for open end
               'missing_start' : r'^[A-Z]+\d+:[A-Z]+$',
               # re for open beginning
               'missing_end' : r'^[A-Z]+:[A-Z]+\d+$'}

def normalize_range(range_string, worksheet) :
    re_strings = [
                # re for at all valid
                r'^[A-Z]+\d*(:[A-Z]+\d*)?$', 
                # re for complete range
                r'^[A-Z]+\d+:[A-Z]+\d+$',
                # re for single cell
                r'^[A-Z]+\d+$',
                # re for only letters
                r'^[A-Z]+:[A-Z]+$',
                # re for open end
                r'^[A-Z]+\d+:[A-Z]+$',
                # re for open beginning
                r'^[A-Z]+:[A-Z]+\d+$']
    res = [re.compile(re_string) for re_string in re_strings]
    if not res[0].match(range_string) :
        raise RuntimeError('This is not a valid descriptor of ranges or cells')
    if res[1].match(range_string) or res[2].match(range_string) :
        return range_string
    if res[5].match(range_string) :
        pos_colon = range_string.find(':')
        return range_string[0:pos_colon] + '1' + range_string[pos_colon:]
    if res[4].match(range_string) :
        return range_string + str(worksheet.row_count)
    if res[3].match(range_string) :
        pos_colon = range_string.find(':')
        return range_string[0:pos_colon] + '1' + range_string[pos_colon:] + str(worksheet.row_count)
    
# def get_rowcol_range(range_string, worksheet) :

In [6]:
def read_gsheet_into_df(range_string, worksheet, date_col=None) :
    range_string = normalize_range(range_string, worksheet)
    cell_range = worksheet.range(range_string)
    result = pd.DataFrame()
    for cell in cell_range :
        if pd.isnull(cell.numeric_value) :
            if cell.value == '' :
                value = cell.numeric_value
            else :
                value = cell.value
        else :
            value = round(cell.numeric_value, 2)
        result.loc[cell.row, num_to_col_letters(cell.col)] = value
        
    if date_col is not None and date_col in result.columns :
        result[date_col] = pd.to_datetime('1899-12-30') + pd.to_timedelta(result[date_col], 'D')
    return result.dropna(how='all')

In [7]:
def read_gsheet_into_df(range_string, worksheet, date_col=None) :
    range_string = normalize_range(range_string, worksheet)
    gspread.models.
            
    if date_col is not None and date_col in result.columns :
        result[date_col] = pd.to_datetime('1899-12-30') + pd.to_timedelta(result[date_col], 'D')
    return result.dropna(how='all')

SyntaxError: invalid syntax (<ipython-input-7-44da3d80670c>, line 3)

In [8]:
workbook = connect_to_sheets()
sheet = connect_to_sheets(workbook, 'August')

In [9]:
col_titles = ['description', 'date', 'amount']

budgeting = read_gsheet_into_df('A6:D', sheet, 'C')
budgeting.columns = ['budget_type'] + col_titles

max_bargeld = read_gsheet_into_df('H7:J', sheet, 'I')
max_bargeld.columns = col_titles
max_bargeld['money_pot'] = 'BM'

paul_bargeld = read_gsheet_into_df('K7:M', sheet, 'L')
paul_bargeld.columns = col_titles
paul_bargeld['money_pot'] = 'BP'

konto = read_gsheet_into_df('N7:P', sheet, 'O')
konto.columns = col_titles
konto['money_pot'] = 'KG'

conjoined = pd.concat([max_bargeld, paul_bargeld, konto])

display(budgeting, conjoined)

,budget_type,description,date,amount
6,R,Miete,2017-08-01,-568.00
7,R,Miete FFM,2017-08-02,-450.00
8,R,Berufsunfähigkeitsversicherung,2017-08-01,-49.05
9,R,Strom EnviaM,NaT,-51.00
10,R,Vodafone,NaT,-19.99
11,M,Telefonie,NaT,-6.00
12,R,Rechtsschutzversicherung,2017-01-30,-13.90
13,R,Haftpflichtversicherung,2017-08-01,-7.50
14,G,GEW,NaT,-2.50
15,R,Semestergebühr Paul,2016-06-21,-46.15


,description,date,amount,money_pot
7,Geld aufladen Mitarbeiter ausweis,2017-08-01,-20.00,BM
8,Geld abheben,2017-08-03,30.00,BM
9,Geld aufladen Trianon,2017-08-02,-20.00,BM
10,Geld aus Schatulle,2017-08-05,1.30,BM
11,Betriebsausflug,2017-08-07,-18.50,BM
12,Geldfund,2017-08-12,0.05,BM
13,Eisessen,2017-08-13,-4.80,BM
14,Wasser,2017-08-13,-1.30,BM
15,BBk Karte Aufladen,2017-08-14,-10.00,BM
16,Eisessen,2017-08-17,-1.20,BM


In [12]:
all_info = pd.merge(budgeting, conjoined, how='outer', 
                    on=['description', 'date', 'amount'])

display(all_info)

perfect_result = all_info.dropna()
display(perfect_result)

,budget_type,description,date,amount,money_pot
0,R,Miete,2017-08-01,-568.00,KG
1,R,Miete FFM,2017-08-02,-450.00,KG
2,R,Berufsunfähigkeitsversicherung,2017-08-01,-49.05,KG
3,R,Strom EnviaM,NaT,-51.00,NaN
4,R,Vodafone,NaT,-19.99,NaN
5,M,Telefonie,NaT,-6.00,NaN
6,R,Rechtsschutzversicherung,2017-01-30,-13.90,NaN
7,R,Haftpflichtversicherung,2017-08-01,-7.50,KG
8,G,GEW,NaT,-2.50,NaN
9,R,Semestergebühr Paul,2016-06-21,-46.15,NaN


,budget_type,description,date,amount,money_pot
0,R,Miete,2017-08-01,-568.00,KG
1,R,Miete FFM,2017-08-02,-450.00,KG
2,R,Berufsunfähigkeitsversicherung,2017-08-01,-49.05,KG
7,R,Haftpflichtversicherung,2017-08-01,-7.50,KG
11,R,Handy Max,2017-08-10,-7.99,KG
17,G,Fitnessstudio,2017-08-03,-39.80,KG
18,L,Aldi,2017-07-31,-23.97,KG
19,R,Monatskarte FFM,2017-07-31,-87.40,KG
20,L,Netto,2017-07-31,-9.44,KG
21,L,Rewe,2017-08-01,-8.71,KG


In [14]:
import sqlite3 as sql
db = sql.connect('my-budget-v0.sqlite')

In [26]:
def id_generation(date) :
    date_int = int(date.strftime('%Y%m%d'))
    crsr = db.cursor()
    crsr.execute('SELECT id FROM money_events WHERE id BETWEEN {} AND {}'.format(
        date_int, date_int + 99))
    results = [row[0] for row in crsr.fetchall()]

    current_id = date_int + 1
    while current_id in results :
        current_id += 1
    if current_id > date_int + 99 :
        raise IndexError('Encountered to many ids for the date {}'.format(date))

    return current_id

In [28]:
def simple_payment(date, description, amount, budget_type, transaction_type, money_pot): 
    new_id = id_generation(date)
    crsr = db.cursor()

    crsr.execute("INSERT INTO money_events VALUES ({}, '{}', '{}', DATE('{}'), NULL, NULL, 'auto-gen');".format(
        new_id, transaction_type, description, date))
    crsr.execute("INSERT INTO budget_events VALUES ({}, '{}', {}, NULL, DATE('{}'), NULL, NULL, 'auto-gen');".format(
        new_id, budget_type, amount, date))
    crsr.execute("INSERT INTO payments VALUES ({}, '{}', {}, NULL, NULL, DATE('{}'), NULL, NULL, 'auto-gen');".format(
        new_id, money_pot, amount, date))

    db.commit()

In [29]:
for index, row in perfect_result.iterrows() :
    simple_payment(row['date'], row['description'], row['amount'], row['budget_type'], 
                   'Kartenzahlung' if row['money_pot'] == 'KG' else 'Barzahlung', row['money_pot'])

2017-08-01 00:00:00
2017-08-02 00:00:00
2017-08-01 00:00:00
2017-08-01 00:00:00
2017-08-10 00:00:00
2017-08-03 00:00:00
2017-07-31 00:00:00
2017-07-31 00:00:00
2017-07-31 00:00:00
2017-08-01 00:00:00
2017-08-01 00:00:00
2017-08-01 00:00:00
2017-08-01 00:00:00
2017-08-01 00:00:00
2017-08-02 00:00:00
2017-08-04 00:00:00
2017-08-05 00:00:00
2017-08-05 00:00:00
2017-08-05 00:00:00
2017-08-05 00:00:00
2017-08-06 00:00:00
2017-08-06 00:00:00
2017-08-07 00:00:00
2017-08-07 00:00:00
2017-08-07 00:00:00
2017-08-07 00:00:00
2017-08-07 00:00:00
2017-08-06 00:00:00
2017-08-13 00:00:00
2017-08-13 00:00:00
2017-08-11 00:00:00
2017-08-11 00:00:00
2017-08-13 00:00:00


In [30]:
def print_action_tables() :
    crsr = db.cursor()

    for table in ['money_events', 'budget_events', 'payments'] :
        display(pd.read_sql_query('SELECT * FROM {};'.format(table), db))
    
print_action_tables()

,id,type,description,creation_date,modification_dates,comments,complete
0,20170732,Kartenzahlung,Aldi,2017-07-31,None,None,YES
1,20170733,Kartenzahlung,Monatskarte FFM,2017-07-31,None,None,YES
2,20170734,Kartenzahlung,Netto,2017-07-31,None,None,YES
3,20170802,Kartenzahlung,Miete,2017-08-01,None,None,YES
4,20170803,Kartenzahlung,Miete FFM,2017-08-02,None,None,YES
5,20170804,Kartenzahlung,Berufsunfähigkeitsversicherung,2017-08-01,None,None,YES
6,20170805,Kartenzahlung,Haftpflichtversicherung,2017-08-01,None,None,YES
7,20170806,Kartenzahlung,Fitnessstudio,2017-08-03,None,None,YES
8,20170807,Kartenzahlung,Rewe,2017-08-01,None,None,YES
9,20170808,Kartenzahlung,Rossmann,2017-08-01,None,None,YES


,id,budget_pot,amount,additional_description,creation_date,modification_dates,comments,complete
0,20170802,R,-568.00,None,2017-08-01,None,None,YES
1,20170803,R,-450.00,None,2017-08-02,None,None,YES
2,20170804,R,-49.05,None,2017-08-01,None,None,YES
3,20170805,R,-7.50,None,2017-08-01,None,None,YES
4,20170811,R,-7.99,None,2017-08-10,None,None,YES
5,20170806,G,-39.80,None,2017-08-03,None,None,YES
6,20170732,L,-23.97,None,2017-07-31,None,None,YES
7,20170733,R,-87.40,None,2017-07-31,None,None,YES
8,20170734,L,-9.44,None,2017-07-31,None,None,YES
9,20170807,L,-8.71,None,2017-08-01,None,None,YES


,id,money_pot,amount,additional_description,budget_effect_date,creation_date,modification_dates,comments,complete
0,20170802,KG,-568.00,None,None,2017-08-01,None,None,YES
1,20170803,KG,-450.00,None,None,2017-08-02,None,None,YES
2,20170804,KG,-49.05,None,None,2017-08-01,None,None,YES
3,20170805,KG,-7.50,None,None,2017-08-01,None,None,YES
4,20170811,KG,-7.99,None,None,2017-08-10,None,None,YES
5,20170806,KG,-39.80,None,None,2017-08-03,None,None,YES
6,20170732,KG,-23.97,None,None,2017-07-31,None,None,YES
7,20170733,KG,-87.40,None,None,2017-07-31,None,None,YES
8,20170734,KG,-9.44,None,None,2017-07-31,None,None,YES
9,20170807,KG,-8.71,None,None,2017-08-01,None,None,YES
